In [ ]:
import cv2, os, warnings, imageio
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec

from types import SimpleNamespace
from scipy import misc

%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
# GLOBAL CAR FILENAME
CAR = '1C4PJLCX8KD22313104.jpeg'

In [ ]:
# Display the Original Image
filename = './images/original/{0}'.format(CAR)
# image = cv2.imread(filename)
img = mpimg.imread(filename) 
plt.rcParams['figure.dpi'] = 200
plt.imshow(img)
plt.show()

In [ ]:
# # Salient Object Detection - Alpha Mask
g_mean = np.array(([126.88,120.24,112.19])).reshape([1,1,3])
OUTPUT = "./images/alpha"
def rgba2rgb(img):
    return img[:,:,:3]*np.expand_dims(img[:,:,3],2)
def make_saliency_alpha(args):
    with tf.Session(config=tf.ConfigProto()) as sess:
        saver = tf.train.import_meta_graph('./salient/graph/my-model.meta')
        saver.restore(sess,tf.train.latest_checkpoint('./salient/model'))
        image_batch = tf.get_collection('image_batch')[0]
        pred_mattes = tf.get_collection('mask')[0]

        rgb = imageio.imread(args.rgb)

        if rgb.shape[2]==4:
            rgb = rgba2rgb(rgb)
        origin_shape = rgb.shape[:2]
        rgb = np.expand_dims(misc.imresize(rgb.astype(np.uint8),[320,320,3], interp="bicubic").astype(np.float32)-g_mean,0)

        feed_dict = {image_batch:rgb}
        pred_alpha = sess.run(pred_mattes,feed_dict = feed_dict)
        final_alpha = misc.imresize(np.squeeze(pred_alpha),origin_shape)
        final_name = '{0}.png'.format(args.car)
        imageio.imwrite(os.path.join(OUTPUT,final_name),final_alpha)

In [ ]:
# Create the alpha mask
INPUT = "./images/original/{0}".format(CAR)
d = {'rgb': INPUT, 'car':CAR}
n = SimpleNamespace(**d)
make_saliency_alpha(n)
# Display the Alpha
filename = './images/alpha/{0}.png'.format(CAR)
image = cv2.imread(filename)
plt.rcParams['figure.dpi'] = 200
plt.imshow(image)
plt.show()

In [ ]:
def make_image(car_name):
    # Read the images
    foreground = mpimg.imread("./images/original/{0}".format(car_name))
    background = cv2.imread("./images/Background1920.jpeg")
    alpha = cv2.imread("./images/alpha/{0}.png".format(car_name))

    # Convert uint8 to float
    foreground = foreground.astype(float)
    background = background.astype(float)

    # Normalize the alpha mask to keep intensity between 0 and 1
    alpha = alpha.astype(float)/255

    # Multiply the foreground with the alpha matte
    foreground = cv2.multiply(alpha, foreground)

    # Multiply the background with ( 1 - alpha )
    background = cv2.multiply(1.0 - alpha, background)

    # Add the masked foreground and background.
    outImage = cv2.add(foreground, background)
    
    out_folder = './images/generated'
    fname = car_name.replace('jpeg','')
    out_filename = '{0}.png'.format(fname)
    imageio.imwrite(os.path.join(out_folder,out_filename),outImage)

In [ ]:
make_image(CAR)

In [ ]:
# Display the Alpha
fname = CAR.replace('jpeg','')
filename = './images/generated/{0}.png'.format(fname)
image = mpimg.imread(filename)
                     
plt.rcParams['figure.dpi'] = 200
plt.imshow(image)
plt.show()